In [9]:
import pandas as pd
import requests
import numpy as np
import yfinance as yf
%matplotlib inline
import random
import plotly.io as pio

import datetime as dt
import plotly.graph_objects as go

import os
from dotenv import load_dotenv

import datetime as dt
from datetime import timedelta

from stable_baselines3 import PPO
import re

import streamlit as st

from diskcache import Cache

pio.templates["custom"] = pio.templates["plotly"]
pio.templates["custom"].layout.font.family = "Cardo"

font_family = "Cardo"

# Set the default template
pio.templates.default = "custom"

In [10]:
load_dotenv()

True

In [11]:
COINGEKCO_KEY = os.getenv('COINGEKCO_KEY')

In [12]:
def normalize_log_returns(log_returns_df, start_date, end_date, normalize_value=1e4):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter the data based on the start date and end date
    filtered_data = log_returns_df[(log_returns_df.index >= start_date) & 
                                   (log_returns_df.index <= end_date)].copy()
    
    if filtered_data.empty:
        print("Filtered data is empty after applying the date filter.")
        return pd.DataFrame()

    # Initialize normalized values
    normalized_values = [normalize_value]
    dates = [filtered_data.index[0]]  # Start date
    versions = [filtered_data['version'].iloc[0]]  # Track the first version

    # Compute normalized returns
    for timestamp, log_return, version in zip(filtered_data.index, filtered_data['Return'], filtered_data['version']):
        normalized_values.append(normalized_values[-1] * np.exp(log_return))
        dates.append(timestamp)
        versions.append(version)  # Track the version used at each timestamp

    # Create DataFrame
    normalized_returns_df = pd.DataFrame({
        'Normalized_Return': normalized_values[1:],  # Exclude initial value
        'version': versions[1:]  # Exclude initial version
    }, index=dates[1:])

    return normalized_returns_df

In [13]:
os.chdir('..')

In [14]:
os.listdir()

['.env',
 '.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'ABIS',
 'aggregate_returns.py',
 'AI_Models',
 'apps',
 'cache_storage',
 'classifier_endpoint.py',
 'data',
 'global_cache',
 'global_classifier_cache',
 'img',
 'live_backend_v2.py',
 'live_results',
 'live_test_cache',
 'live_v01_cache',
 'mint_shares.py',
 'models',
 'notebooks',
 'notes',
 'onchain apps',
 'python_scripts',
 'README.md',
 'requirements.txt',
 'results',
 'sql_scripts',
 'steth_optimizer',
 'templates',
 'test_model_cache',
 'Vault-Contracts',
 '__pycache__']

In [15]:
live_results_path = r'E:\Projects\portfolio_optimizers\classifier_optimizer\live_results'

In [16]:
file_list = os.listdir('E:\Projects\portfolio_optimizers\classifier_optimizer\live_results')

model_names = sorted(set(re.findall(r'v\d{2}', ' '.join(file_list))))

print(model_names)

['v01']


In [17]:
model_data = pd.DataFrame()

for model_name in model_names:
    values = pd.read_csv(f'E:\Projects\portfolio_optimizers\classifier_optimizer\live_results\{model_name}.csv')
    values['version'] = model_name

    model_data = pd.concat([model_data,values])

model_data


,index,Return,version
0,2025-02-28 13:00:00,0.000000,v01
1,2025-02-28 14:00:00,0.001750,v01
2,2025-02-28 15:00:00,0.024522,v01
3,2025-02-28 16:00:00,0.025883,v01
4,2025-02-28 17:00:00,0.009480,v01
5,2025-02-28 18:00:00,-0.015425,v01
6,2025-02-28 19:00:00,0.003622,v01
7,2025-02-28 20:00:00,0.014006,v01
8,2025-02-28 21:00:00,0.038282,v01
9,2025-02-28 22:00:00,-0.025000,v01


In [33]:
from python_scripts.utils import mvo, calculate_log_returns, calculate_portfolio_returns
from chart_builder.scripts.utils import data_processing

In [48]:

oracle_prices_copy = data_processing(path=r'E:\Projects\portfolio_optimizers\classifier_optimizer\data\oracle_prices.csv',to_clean_dates=False)

turning to df
                  hour  FRAX_Price  HEGIC_Price  SPA_Price
0  2025-02-28 08:00:00    0.995879     0.024834   0.016066
1  2025-02-28 09:00:00    0.995890     0.024839   0.016019
2  2025-02-28 10:00:00    0.996099     0.024863   0.016240
3  2025-02-28 11:00:00    0.996077     0.024792   0.016261
4  2025-02-28 12:00:00    0.996219     0.024765   0.016099 
                    hour  FRAX_Price  HEGIC_Price  SPA_Price
28  2025-03-01 12:00:00    0.995890     0.025859   0.019535
29  2025-03-01 13:00:00    0.995835     0.025668   0.018849
30  2025-03-01 14:00:00    0.995873     0.025632   0.019227
31  2025-03-01 15:00:00    0.995921     0.025516   0.019118
32  2025-03-01 16:00:00    0.995921     0.025516   0.019118
Dropping Duplicates
turn to time: True
set time col: False
turning to dt
None
['date', 'dt', 'hour', 'time', 'day', 'month', 'year', 'week', 'timestamp', 'date(utc)', 'block_timestamp', 'ds', 'period', 'date_time', 'trunc_date', 'quarter', 'block_time', 'block_date', No

In [49]:


hist_comp = data_processing(path=r'E:\Projects\portfolio_optimizers\classifier_optimizer\data\live_model_comp.csv',to_clean_dates=False)

turning to df
   Unnamed: 0    HEGIC comp  SPA comp  FRAX comp     WETH comp  \
0           0  5.581770e-12       1.0        0.0  4.939648e-17   
1           1  5.620441e-12       1.0        0.0  5.019835e-17   
2           2  5.608132e-12       1.0        0.0  5.042642e-17   
3           3  5.469036e-12       1.0        0.0  5.044244e-17   
4           4  5.331741e-12       1.0        0.0  4.921908e-17   

                  date  
0  2025-02-28 13:00:00  
1  2025-02-28 14:00:00  
2  2025-02-28 15:00:00  
3  2025-02-28 16:00:00  
4  2025-02-28 17:00:00   
     Unnamed: 0    HEGIC comp  SPA comp  FRAX comp     WETH comp  \
22          22  4.736948e-12       1.0        0.0  4.212637e-17   
23          23  4.885630e-12       1.0        0.0  4.222969e-17   
24          24  4.846167e-12       1.0        0.0  4.209864e-17   
25          25  4.880690e-12       1.0        0.0  4.238862e-17   
26          27  4.886158e-12       1.0        0.0  4.289315e-17   

                   date  
22  2025

In [50]:
price_returns = calculate_log_returns(oracle_prices_copy[oracle_prices_copy.index>=hist_comp.index.min()])


In [54]:
SPA_return = price_returns[['SPA_Price']]

In [57]:
hist_comp = hist_comp[['SPA comp']]

In [61]:
df = pd.merge(hist_comp,SPA_return,left_index=True,right_index=True,how='outer' )
df

,SPA comp,SPA_Price
2025-02-28 13:00:00,1.0,0.000000
2025-02-28 14:00:00,1.0,0.001750
2025-02-28 15:00:00,1.0,0.024522
2025-02-28 16:00:00,1.0,0.025883
2025-02-28 17:00:00,1.0,0.009480
2025-02-28 18:00:00,1.0,-0.015425
2025-02-28 19:00:00,1.0,0.003622
2025-02-28 20:00:00,1.0,0.014006
2025-02-28 21:00:00,1.0,0.038282
2025-02-28 22:00:00,1.0,-0.025000


In [59]:
hist_returns = calculate_portfolio_returns(hist_comp, price_returns)
hist_returns = hist_returns.to_frame('Portfolio Return')
analysis_df = hist_returns.merge(df, left_index=True,right_index=True,how='inner')

tokens_price: ['FRAX', 'HEGIC', 'SPA']
tokens_comp: ['SPA']
common_tokens: {'SPA'}
comp_columns: ['SPA comp']
price_columns: ['SPA_Price']
common_index: DatetimeIndex(['2025-02-28 13:00:00', '2025-02-28 14:00:00',
               '2025-02-28 15:00:00', '2025-02-28 16:00:00',
               '2025-02-28 17:00:00', '2025-02-28 18:00:00',
               '2025-02-28 19:00:00', '2025-02-28 20:00:00',
               '2025-02-28 21:00:00', '2025-02-28 22:00:00',
               '2025-02-28 23:00:00', '2025-03-01 00:00:00',
               '2025-03-01 01:00:00', '2025-03-01 02:00:00',
               '2025-03-01 03:00:00', '2025-03-01 04:00:00',
               '2025-03-01 05:00:00', '2025-03-01 06:00:00',
               '2025-03-01 07:00:00', '2025-03-01 08:00:00',
               '2025-03-01 09:00:00', '2025-03-01 10:00:00',
               '2025-03-01 11:00:00', '2025-03-01 12:00:00',
               '2025-03-01 13:00:00', '2025-03-01 15:00:00',
               '2025-03-01 16:00:00'],
              d

In [60]:
analysis_df

,Portfolio Return,SPA comp,SPA_Price
2025-02-28 13:00:00,0.000000,1.0,0.000000
2025-02-28 14:00:00,0.001750,1.0,0.001750
2025-02-28 15:00:00,0.024522,1.0,0.024522
2025-02-28 16:00:00,0.025883,1.0,0.025883
2025-02-28 17:00:00,0.009480,1.0,0.009480
2025-02-28 18:00:00,-0.015425,1.0,-0.015425
2025-02-28 19:00:00,0.003622,1.0,0.003622
2025-02-28 20:00:00,0.014006,1.0,0.014006
2025-02-28 21:00:00,0.038282,1.0,0.038282
2025-02-28 22:00:00,-0.025000,1.0,-0.025000


In [53]:
model_data['index'] = pd.to_datetime(model_data['index'])
model_data.set_index('index',inplace=True)


In [ ]:
filled_data = model_data.resample('h').agg({
    "Return":'last',
    "version":'last'
})

In [ ]:
norm_model_returns = normalize_log_returns(filled_data, filled_data.index.min(), filled_data.index.max(),100)

In [ ]:
def plot_continuous_return_with_versions(df, title="Portfolio Return Over Time"):
    """Plots a continuous normalized return line but visually separates different model versions."""

    if df.empty:
        raise ValueError("DataFrame is empty. Please provide valid data.")

    if 'Normalized_Return' not in df.columns or 'version' not in df.columns:
        raise ValueError("DataFrame must contain 'Normalized_Return' and 'version' columns.")

    # Ensure data is sorted by time
    df = df.sort_index()

    # Forward-fill missing values for continuity
    df['Normalized_Return'] = df['Normalized_Return'].ffill()
    df['version'] = df['version'].fillna("Unknown")  # Ensure no NaN values in 'version'

    # Create figure
    fig = go.Figure()

    # Get unique versions in order of appearance
    unique_versions = df['version'].unique()
    print(F'unique_versions: {unique_versions}')

    # Identify version change points
    version_change_points = df['version'] != df['version'].shift(1)
    version_change_times = df.index[version_change_points]

    # Avoid adding dashed lines if there's only one version
    add_dashed_lines = len(unique_versions) > 1

    # Loop through each version and plot it as a segment
    for idx, version in enumerate(unique_versions):
        version_df = df[df['version'] == version].copy()

        if version_df.empty:
            continue  # Skip empty segments

        # Ensure continuity: Include the last value of the previous version
        if idx > 0:
            prev_version_df = df[df['version'] == unique_versions[idx - 1]]
            if not prev_version_df.empty:
                last_old_value = prev_version_df.iloc[-1:].copy()
                version_df = pd.concat([last_old_value, version_df])

        # Define color for this version
        color = f"hsl({(idx * 50) % 360}, 70%, 50%)"  # Generate distinct colors

        # Plot version as part of a single continuous line
        fig.add_trace(go.Scatter(
            x=version_df.index,
            y=version_df['Normalized_Return'],
            mode='lines',  # Only lines to keep smooth transitions
            line=dict(color=color, width=4),
            name=f"Version {version}"
        ))

    # Add vertical dashed lines at version change points (only if there are multiple versions)
    if add_dashed_lines:
        for version_time, version in zip(version_change_times, df['version'][version_change_points]):
            fig.add_shape(
                go.layout.Shape(
                    type="line",
                    x0=version_time, x1=version_time,
                    y0=df['Normalized_Return'].min(), y1=df['Normalized_Return'].max(),
                    line=dict(color="black", width=2, dash="dash")  # Dashed vertical line
                )
            )

            # Add version annotation near the top of the plot
            fig.add_annotation(
                x=version_time, 
                y=df['Normalized_Return'].max(),
                text=f"Version {version}",
                showarrow=False,
                font=dict(size=14, color="black"),
                yshift=10
            )

    # Customize layout
    fig.update_layout(
        title=title,
        xaxis_title="Time",
        yaxis_title="Normalized Return",
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        legend_title="Model Versions",
        template="plotly_white"
    )

    return fig


In [ ]:
norm_model_returns

In [ ]:
fig = plot_continuous_return_with_versions(norm_model_returns)
fig.show()

In [ ]:
os.chdir('..')

In [ ]:
base_cache_dir = r'E:\Projects\portfolio_optimizers\classifier_optimizer'
global_classifier_cache = Cache(os.path.join(base_cache_dir, 'global_classifier_cache'))

In [ ]:
import requests
import json

url = "https://api.coingecko.com/api/v3/coins/defipulse-index/market_chart?vs_currency=usd&days=365"

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": COINGEKCO_KEY
}

response = requests.get(url, headers=headers)
response_text = response.text
data = json.loads(response_text)
df_prices = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
df_prices["timestamp"] = pd.to_datetime(df_prices["timestamp"], unit='ms')  # Convert to datetime
df_prices.set_index('timestamp',inplace=True)
model_name = global_classifier_cache.get('current_model_name')
df = pd.read_csv(f'E:/Projects/portfolio_optimizers/classifier_optimizer/results/{model_name}/norm_returns.csv')
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.set_index('Unnamed: 0',inplace=True)
daily_df = df.resample('D').last()
daily_df.index = pd.to_datetime(daily_df.index.strftime('%Y-%m-%d'))
df_prices.rename(columns={'price':'DPI Price'},inplace=True)
daily_df.rename(columns={"Return":"Portfolio Return"},inplace=True)
analysis_df = pd.merge(
    df_prices,
    daily_df,
    left_index=True,
    right_index=True,
    how='inner'
)

In [ ]:
analysis_df

In [ ]:
from python_scripts.utils import calculate_cagr, calculate_beta

In [ ]:
dpi_cagr = calculate_cagr(analysis_df['DPI Price'])
current_risk_free = 0.047
dpi_cumulative_risk_premium = dpi_cagr - current_risk_free
portfolio_cagr = calculate_cagr(analysis_df['Portfolio Return'])
portfolio_beta = calculate_beta(analysis_df, 'DPI Price'
                                ,'Portfolio Return')
portfolio_expected_return = current_risk_free + (portfolio_beta*dpi_cumulative_risk_premium)
f'{portfolio_expected_return * 100:.2f}%'

# Expected return over one year (annualzied)

Cost of Equity (14.98%) indicates the annualized rate of return required by investors for holding equity in this portfolio.
It represents the compensation investors expect for the risk they take by investing in this portfolio instead of a risk-free asset.

In [ ]:
portfolio_expected_return

In [ ]:
def calculate_expected_return():
    url = "https://api.coingecko.com/api/v3/coins/defipulse-index/market_chart?vs_currency=usd&days=365"

    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": COINGEKCO_KEY
    }

    response = requests.get(url, headers=headers)
    response_text = response.text
    data = json.loads(response_text)
    df_prices = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
    df_prices["timestamp"] = pd.to_datetime(df_prices["timestamp"], unit='ms')  # Convert to datetime
    df_prices.set_index('timestamp',inplace=True)
    model_name = global_classifier_cache.get('current_model_name')
    df = pd.read_csv(f'E:/Projects/portfolio_optimizers/classifier_optimizer/results/{model_name}/norm_returns.csv')
    df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('Unnamed: 0',inplace=True)
    daily_df = df.resample('D').last()
    daily_df.index = pd.to_datetime(daily_df.index.strftime('%Y-%m-%d'))
    df_prices.rename(columns={'price':'DPI Price'},inplace=True)
    daily_df.rename(columns={"Return":"Portfolio Return"},inplace=True)
    analysis_df = pd.merge(
        df_prices,
        daily_df,
        left_index=True,
        right_index=True,
        how='inner'
    )
    dpi_cagr = calculate_cagr(analysis_df['DPI Price'])
    # current_risk_free = 0.047
    dpi_cumulative_risk_premium = dpi_cagr - current_risk_free
    portfolio_cagr = calculate_cagr(analysis_df['Portfolio Return'])
    portfolio_beta = calculate_beta(analysis_df, 'DPI Price'
                                    ,'Portfolio Return')
    portfolio_expected_return = current_risk_free + (portfolio_beta*dpi_cumulative_risk_premium)
    f'{portfolio_expected_return * 100:.2f}%'
    
    return portfolio_expected_return
    